## Deploy Metadata Extraction from Documents Model Package from AWS Marketplace 


Digital content in different file formats like pdfs, docs, ppts, emails have a lot of metadata as well as formatting data associated with them. This solution parses documents, extracts metadata and formatting information from documents and standardizes them. It provides information about the document whether it is password protected, encrypted etc. along with the formatting information like Highlights, List, Section and Sub-Section etc. at sentence level. These details help in augmenting NLP algorithms with added information for decision making.

This sample notebook shows you how to deploy Metadata Extraction from Documents using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Metadata Extraction from Documents. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Metadata Extraction from Documents
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/metadataextraction-copy-03-08'

In [2]:
import pandas as pd
import json
import os
import boto3
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image
from PIL import Image as ImageEdit

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='meta-data-extraction'

content_type='application/pdf'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [6]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

##### Input Instructions

•	The input must be a pdf file

•	Maximum input file sizes : 5Mb


In [7]:
# To show pdf-s inside ipython/jupyter notebooks you can use IFrame
from IPython.display import IFrame
IFrame("./Input/sample.pdf", width=600, height=300)

In [9]:
file_name = "./Input/sample.pdf"
output_file_name = 'output.json'

#### C. Perform real-time inference

In [10]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'meta-data-extraction' \
    --body fileb://$file_name \
    --content-type 'application/pdf' \
    --region us-east-2 \
    output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

The processed output is of the form .csv file with following columns :
    
    Entity: This column will contain the original entity from the text data.
    
    Entity_Label: This columns will contain the named entity recognized with corresponding entity.
    
    index: This columns will contain number representing the order in which entity appears in the text data.

In [11]:
import json
with open('./output.json') as f:
    data = json.load(f)

print("Output: ")
print(data.keys())

print(data["meta_data"])

Output: 
dict_keys(['meta_data', 'formatting'])
{'Content-Type': 'application/pdf', 'Creation-Date': '2021-05-20T20:28:02Z', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToXMLContentHandler', 'X-TIKA:embedded_depth': '0', 'X-TIKA:parse_time_millis': '1825', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_degraded': 'true', 'access_permission:extract_content': 'true', 'access_permission:extract_for_accessibility': 'true', 'access_permission:fill_in_form': 'true', 'access_permission:modify_annotations': 'true', 'created': '2021-05-20T20:28:02Z', 'dc:format': 'application/pdf; version=1.7', 'dcterms:created': '2021-05-20T20:28:02Z', 'meta:creation-date': '2021-05-20T20:28:02Z', 'pdf:PDFVersion': '1.7', 'pdf:charsPerPage': ['72', '40', '1515', '573', '1115', '1032', '4690', '4364', '5313', '5206', '2814', '

In [12]:
data["formatting"]

{'1': [{'sent_id': 1,
   'sentence': 'Mphasis Limited | Annual Report 2018',
   'font': 'Lt',
   'is_caps': 0,
   'colour': ['1', '1', '1'],
   'colourspace': 'ICCBased',
   'size': 18.896,
   'is_topic': 0,
   'is_highlight': 0,
   'is_copyright': 0,
   'is_subtopic': 0,
   'subtopic_id': None,
   'is_list': 0,
   'url_present': 0,
   'email_present': 0,
   'batch_no_pattern': 0},
  {'sent_id': 2,
   'sentence': 'Mphasis Limited | Annual Report 2018',
   'font': 'Lt',
   'is_caps': 0,
   'colour': ['1', '1', '1'],
   'colourspace': 'ICCBased',
   'size': 18.896,
   'is_topic': 0,
   'is_highlight': 0,
   'is_copyright': 0,
   'is_subtopic': 0,
   'subtopic_id': None,
   'is_list': 0,
   'url_present': 0,
   'email_present': 0,
   'batch_no_pattern': 0}],
 '2': [{'sent_id': 2,
   'sentence': 'GROWTH APPLIED',
   'font': 'Lt',
   'is_caps': 1,
   'colour': ['1', '1', '1'],
   'colourspace': 'ICCBased',
   'size': 33.068,
   'is_topic': 0,
   'is_highlight': 0,
   'is_copyright': 0,
   '

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [13]:
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [14]:
#upload the batch-transform job input files to S3
transform_input_folder = "Input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/meta-data-extraction


In [15]:
#Run the batch-transform job
transformer = model.transformer(1, 'ml.m5.4xlarge')
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

.....................2021-05-20T15:22:02.248:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
/opt/program/tika/tika-server-1.9.jar
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat
/opt/program/tika/tika-server-1.9.jar
 * Debugger is active!
 * Debugger PIN: 156-672-190
169.254.255.130 - - [20/May/2021 15:22:02] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [20/May/2021 15:22:02] "#033GET /execution-parameters HTTP/1.1#033" 404 -
input/sample.pdf
2021-05-20 15:22:02,683 [Thread-4    ] [WARNI]  Failed to see startup log message; retrying...
/opt/program/tika/tika-server-1.9.jar
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting w

In [16]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/metadataextraction-copy-03-08-2021-05-20-15-18-42-898'

In [17]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('./output.json', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, transformer.output_path.split("/")[-1]+'/sample.pdf.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [18]:
output = open("./output.json","r+")
#print(output.read())

# copy and paste the output below in a browser tab to see the result
data = output.read()
print(data)

{"meta_data": {"Content-Type": "application/pdf", "Creation-Date": "2021-05-20T20:28:02Z", "X-Parsed-By": ["org.apache.tika.parser.DefaultParser", "org.apache.tika.parser.pdf.PDFParser"], "X-TIKA:content_handler": "ToXMLContentHandler", "X-TIKA:embedded_depth": "0", "X-TIKA:parse_time_millis": "1289", "access_permission:assemble_document": "true", "access_permission:can_modify": "true", "access_permission:can_print": "true", "access_permission:can_print_degraded": "true", "access_permission:extract_content": "true", "access_permission:extract_for_accessibility": "true", "access_permission:fill_in_form": "true", "access_permission:modify_annotations": "true", "created": "2021-05-20T20:28:02Z", "dc:format": "application/pdf; version=1.7", "dcterms:created": "2021-05-20T20:28:02Z", "meta:creation-date": "2021-05-20T20:28:02Z", "pdf:PDFVersion": "1.7", "pdf:charsPerPage": ["72", "40", "1515", "573", "1115", "1032", "4690", "4364", "5313", "5206", "2814", "4659", "1548", "1056", "2066", "18

### 4. Clean-up

#### A. Delete the model

In [19]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.